In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


In [25]:
# Device Setting
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('mps') # M1 silicon 사용
device

device(type='mps')

In [26]:
# Seed setting
import random

def def_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

def_seed(seed=123)

In [27]:
# Hyperparameters for Neural Network
epochs = 100
lr = 0.01
batch_size = 4096

In [4]:
# Data Load
df_train = pd.read_pickle('train.pkl')
df_test = pd.read_pickle('test.pkl')

In [5]:
df_train.head(3)

,Datetime,ADID_type,DSP_ID,Media_ID,Adunit_ID,Platform,OS_type,Size_ID,Ex_Rate,Country_ID,...,IAB_18,IAB_19,IAB_20,IAB_21,IAB_22,IAB_23,IAB_24,IAB_25,IAB_26,Class
0,2022-04-03,1,7,13,296,1,1,3,1218.0,1012,...,0,0,0,0,1,0,0,0,0,0
1,2022-04-03,1,7,13,296,1,1,3,1218.0,1012,...,0,0,0,0,1,0,0,0,0,0
2,2022-04-03,1,7,13,296,1,1,3,1218.0,1012,...,0,0,0,0,1,0,0,0,0,0


## Dataset Setting

In [16]:
class MyDataset(Dataset):
    def __init__(self, df, if_test):
        self.df = df
        self.if_test = if_test
        if self.if_test:
            self.labels = self.df['winning'].values
            self.df = self.df.drop(columns=['winning']).values
        else:
            self.df = self.df.values

    def __getitem__(self, index):
        if self.if_test:
            self.x = self.df[index]
            self.y = self.labels[index]
            return torch.Tensor(self.x), self.y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x)
    
    def __len__(self):
        return len(self.df)